## Update Data

In [13]:
# %load ../src/data/get_data.py
import subprocess
import os

import numpy as np
import pandas as pd


def get_johns_hopkins():
    git_pull = subprocess.Popen( "/usr/bin/git pull" , 
                     cwd = os.path.dirname( '../data/raw/COVID-19/' ), 
                     shell = True, 
                     stdout = subprocess.PIPE, 
                     stderr = subprocess.PIPE )
    (out, error) = git_pull.communicate()


    print("Error : " + str(error)) 
    print("out : " + str(out))

get_johns_hopkins()

Error : b''
out : b'Already up to date.\n'


## Process Pipeline

In [14]:
# %load ../src/data/process_HJ_data.py
import subprocess
import os

import numpy as np
import pandas as pd

from datetime import datetime

def store_relational_JH_data():
    pd_raw = pd.read_csv('../data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
    pd_data_base = pd_raw.rename(columns={'Country/Region': 'country', 'Province/State': 'state'})
    pd_data_base['state'] = pd_data_base['state'].fillna('no')
    pd_data_base.drop(['Lat', 'Long'], axis=1, inplace=True)
    pd_relational_model = pd_data_base.set_index(['state', 'country']) \
            .T.stack(level=[0, 1]) \
            .reset_index()  \
            .rename(columns = {'level_0': 'date', 0: 'confirmed'},)

    pd_relational_model['date'] = pd_relational_model.date.astype('datetime64[ns]')
    pd_relational_model.to_csv('../data/processed/Covid_relational_confirmed.csv', sep = ';', index=False)
    print('Number of rows stored:'+ str(pd_relational_model.shape[0]))

if __name__ == '__main__':
    store_relational_JH_data()

Number of rows stored:110696


In [16]:
# %load ../src/features/build_features.py
import numpy as np

from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=True)

def get_doubling_time_via_regression(in_array):
    ''' Use a linear regression to approximate the doubling rate'''
    
    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1, 1)
    
    assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_
    
    return intercept/slope

if __name__ == '__main__':
    test_data = np.array([2, 4, 6])
    result = get_doubling_time_via_regression(test_data)
    print('The test slope is: ' + str(result))

The test slope is: [2.]
